In [19]:
import torch
from torch import optim
from torch.nn import functional as F
from torch import nn
from torch.autograd import Variable

import dlc_practical_prologue as prologue

In [98]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [99]:
def train_model(model, train_input, train_target, nb_epochs, mini_batch_size):
    criterion = nn.CrossEntropyLoss()
    eta = 1e-3
    optimizer = optim.SGD(model.parameters(), lr = eta)

    for e in range(nb_epochs):
        acc_loss = 0
        
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            acc_loss = acc_loss + loss.item()
            model.zero_grad()
            loss.backward()
            optimizer.step()
    
        #print(e, acc_loss)

In [127]:
class MultilayerPerceptron(nn.Module):
    def __init__(self):
        super(MultilayerPerceptron, self).__init__()
        
        nb_hidden = 50
        input_size = 2*14*14

        self.layer1 = nn.Linear(input_size, nb_hidden)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(nb_hidden, 2)
        self.layer3 = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = x.view(x.size(0),-1)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
    
        return out

In [128]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)
    
# train_input, train_target, train_classes \
#     = Variable(train_input), Variable(train_target), Variable(train_classes)
# test_input, test_target, test_classes \
#     = Variable(test_input), Variable(test_target), Variable(test_classes)

def get_tests(n):
    M = []
    for k in range (0, n):
        L = []
        _, _, _, test_input, test_target, test_classes =  prologue.generate_pair_sets(1000)
        L.append(test_input)
        L.append(test_target)
        L.append(test_classes)
        M.append(L)
    return M


MLP = MultilayerPerceptron()
nb_epochs = 25
mini_batch_size = 100

train_model(MLP, train_input, train_target, nb_epochs, mini_batch_size)
L = get_tests(10)
average_nb_test_error = 0
for k in range (0, len(L)):
    nb_test_errors = compute_nb_errors(MLP, L[k][0], L[k][1], mini_batch_size)
    average_nb_test_error += nb_test_errors
    print('test error MultilayerPerceptron {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / L[k][0].size(0),
                                                nb_test_errors, L[k][0].size(0)))
print('Average test error MultilayerPerceptron {:0.2f}% {:0.1f}/{:d}'.format((100*average_nb_test_error/10) / L[0][0].size(0),
                                                                  average_nb_test_error/10, L[0][0].size(0) ))

0 73.48645889759064
1 11.078763127326965
2 6.412270665168762
3 5.4333192110061646
4 4.783904492855072
5 4.322046369314194
6 3.9917860329151154
7 3.713072955608368
8 3.477197825908661
9 3.2897271811962128
10 3.114551231265068
11 2.96530582010746
12 2.821431055665016
13 2.6925964802503586
14 2.5848399996757507
15 2.4637518227100372
16 2.3615109771490097
17 2.2755756825208664
18 2.1790389716625214
19 2.097402885556221
20 2.015765607357025
21 1.9431884586811066
22 1.8816437721252441
23 1.8039746582508087
24 1.7442994862794876
test error MultilayerPerceptron 27.20% 272/1000
test error MultilayerPerceptron 27.10% 271/1000
test error MultilayerPerceptron 26.50% 265/1000
test error MultilayerPerceptron 27.60% 276/1000
test error MultilayerPerceptron 27.10% 271/1000
test error MultilayerPerceptron 27.90% 279/1000
test error MultilayerPerceptron 27.90% 279/1000
test error MultilayerPerceptron 28.00% 280/1000
test error MultilayerPerceptron 27.30% 273/1000
test error MultilayerPerceptron 28.40% 2